# TP2  DE MACHINE LEARNING

## PHASE 1 : REGRESSION LINEAIRE AVANCEE ET COMPROMIS

### 1.1 Préparation des données et OLS
Nous utilisons lee dataset Diabetes pour prédire la progression de la maladie.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Tâche 1.1 & 1.2 : Chargement et OLS
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraînement OLS (Ordinary Least Squares)
ols_reg = LinearRegression()
ols_reg.fit(X_train_scaled, y_train)
y_pred_ols = ols_reg.predict(X_test_scaled)

# Évaluation
rmse_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))
r2_ols = r2_score(y_test, y_pred_ols)

print(f"OLS: RMSE={rmse_ols:.2f}, R2={r2_ols:.2f}")
print("Coefficients OLS:", np.round(ols_reg.coef_, 2))

OLS: RMSE=53.85, R2=0.45
Coefficients OLS: [  1.75 -11.51  25.61  16.83 -44.45  24.64   7.68  13.14  35.16   2.35]


### 1.2 Théorie : Biais, Variance et Régularisation

Tâche 1.3

Le Biais : Erreur due à des hypothèses trop simplistes (ex: utiliser un modèle linéaire pour des données complexes). Cela mène au sous-apprentissage (underfitting).

La Variance : Erreur due à une trop grande sensibilité du modèle aux fluctuations du dataset d'entraînement. Cela mène au sur-apprentissage (overfitting).

La Régularisation : En ajoutant une pénalité aux coefficients trop élevés, on accepte un peu plus de biais pour réduire drastiquement la variance, ce qui améliore la généralisation sur de nouvelles données.

Tâche 1.4 : Ridge Regression ($L_2$) 3
La pénalité $L_2$ ajoute la somme des carrés des coefficients à la fonction de perte.Pourquoi elle ne s'annule jamais ? Mathématiquement, la pénalité est une parabole. La dérivée diminue à mesure que le coefficient s'approche de zéro, mais ne "pousse" pas brutalement le coefficient à zéro. Les poids deviennent très petits mais restent non-nuls.

In [2]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# Entrainement avec une valeur alpha initiale
alpha_initial = 1.0
ridge_reg = Ridge(alpha=alpha_initial)
ridge_reg.fit(X_train_scaled, y_train)

# Evaluation by Cross-Validation (5-folds)
scores_ridge = cross_val_score(ridge_reg, X_train_scaled, y_train, scoring='neg_root_mean_squared_error', cv=5)
rmse_cv_ridge = -scores_ridge.mean()

print(f"\nRidge (alpha={alpha_initial}): RMSE CV={rmse_cv_ridge:.2f}")
print("Coefficients Ridge:", ridge_reg.coef_)


Ridge (alpha=1.0): RMSE CV=55.92
Coefficients Ridge: [  1.80734179 -11.44818951  25.73269892  16.73429974 -34.67195409
  17.05307485   3.36991411  11.76426044  31.3783838    2.45813922]


### 1.3 Sparité et optimisation (LASSO)

In [3]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Tâche 1.5 : Implémentation d'un modèle LASSO avec un alpha initial
alpha_lasso_init = 0.1
lasso_reg = Lasso(alpha=alpha_lasso_init, max_iter=10000)

# Entraînement sur les données normalisées
lasso_reg.fit(X_train_scaled, y_train)

# Prédiction
y_pred_lasso = lasso_reg.predict(X_test_scaled)

# Évaluation
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"LASSO (alpha={alpha_lasso_init}): RMSE={rmse_lasso:.2f}, R2={r2_lasso:.2f}")
print("Coefficients LASSO :", lasso_reg.coef_)

LASSO (alpha=0.1): RMSE=53.71, R2=0.46
Coefficients LASSO : [  1.73045056 -11.31635911  25.82462699  16.64425156 -29.35841191
  13.27584411   0.5479479   10.23616805  29.63282611   2.39347521]


Sparsité : Un modèle est dit "creux" (sparse) lorsqu'un grand nombre de ses coefficients sont exactement égaux à zéro.Pourquoi le $L_1$ annule les coefficients ? La forme géométrique de la pénalité $L_1$ (un diamant) possède des coins sur les axes. Lors de l'optimisation, la solution a de fortes chances de toucher un de ces coins, annulant ainsi la variable. C'est une sélection de caractéristiques intrinsèque.

In [5]:
# Tâche 1.6 : Recherche des Hyperparamètres Optimaux
from sklearn.linear_model import RidgeCV, LassoCV

# Tests sur 100 valeurs de alpha
alphas = np.logspace(-3, 2, 100)

ridge_cv = RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(X_train_scaled, y_train)

lasso_cv = LassoCV(alphas=np.logspace(-3, 0, 100), cv=10, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)

print(f"Alpha optimal Ridge : {ridge_cv.alpha_:.4f}")
print(f"Alpha optimal LASSO : {lasso_cv.alpha_:.4f}")

#Final LASSO Model
lasso_final = LassoCV(alphas=[lasso_cv.alpha_], cv=10, max_iter=10000)
lasso_final.fit(X_train_scaled, y_train)

# Analyse de la sparsité du LASSO
print(f"Variables annulées par LASSO : {np.sum(lasso_cv.coef_ == 0)} sur {X.shape[1]}")

Alpha optimal Ridge : 55.9081
Alpha optimal LASSO : 0.1417
Variables annulées par LASSO : 1 sur 10


## PHASE 2 : MLOps ET MISE EN PRODUCTION

### 2.1 Packaging et Dépendances

Tâche 2.1 : Pourquoi sauvegarder le scaler ?  Il est impératif de sauvegarder l'objet StandardScaler car les nouvelles données en production doivent subir exactement la même transformation (même moyenne et même écart-type) que les données d'entraînement. Utiliser un nouveau scaler sur les données de production fausserait totalement les prédictions.

In [11]:
import joblib
import numpy as np

# Tâche 2.1 : Sauvegarde du modèle et du scaler
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(lasso_final, 'lasso_model.pkl')
print("Scaler and LASSO model saved for deployment.")

# Fonction de prédiction corrigée
def predict_new_data(raw_data, model_path='lasso_model.pkl', scaler_path='scaler.pkl'):
    """Simule le pipeline de prédiction en production."""
    try:
        # Chargement des objets
        loaded_scaler = joblib.load(scaler_path)
        loaded_model = joblib.load(model_path)

        # Etape 1: Transformation des données brutes 
        # On s'assure que raw_data est au bon format (2D array pour sklearn)
        data_to_predict = np.array(raw_data).reshape(1, -1)
        scaled_data = loaded_scaler.transform(data_to_predict)

        # Etape 2: Prédiction 
        prediction = loaded_model.predict(scaled_data)[0]
        return prediction

    except FileNotFoundError:
        print("Error: Model or scaler files not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test de la fonction avec un point du jeu de test [cite: 230]
# Utilisation de .iloc[0] si X_test est un DataFrame Pandas
example_raw_input = X_test[0] if isinstance(X_test, np.ndarray) else X_test.iloc[0]
predicted_value = predict_new_data(example_raw_input)

if predicted_value is not None:
    print(f"Prediction for the test example: {predicted_value:.2f}")

Scaler and LASSO model saved for deployment.
Prediction for the test example: 140.05


### 2.2 Tracking avec MLflow

MLflow permet de garder un historique de toutes les expériences pour comparer les modèles objectivement .

In [12]:
pip install mlflow


Note: you may need to restart the kernel to use updated packages.


In [10]:
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Configuration de l'expérience
mlflow.set_experiment("TP_Advanced_Regression")

def log_model_results(model_name, params, metrics, model):
    
    # Enregistre les résultats d'un modèle dans MLflow de façon modulaire.
    
    with mlflow.start_run(run_name=model_name):
        # Hyperparameter 
        mlflow.log_params(params)
        
        # Metric Logging 
        mlflow.log_metrics(metrics)
        
        # Model Logging 
        mlflow.sklearn.log_model(model, "model")
        
        print(f"MLflow Run '{model_name}' enregistré avec succès.")

# Exemple d'application pour le modèle LASSO

# 1. Préparation des paramètres
lasso_params = {
    "alpha": lasso_cv.alpha_, 
    "solver": "cd", 
    "penalty": "l1",
    "selection": "cyclic"
}

# 2. Calcul des métriques sur le test set
y_pred_lasso = lasso_cv.predict(X_test_scaled)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

lasso_metrics = {
    "test_rmse": rmse_lasso, 
    "test_r2": r2_lasso
}

# 3. Appel de la fonction (Structure conforme au Code 2)
log_model_results("LASSO_Optimal", lasso_params, lasso_metrics, lasso_cv)

2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/18 19:27:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/18 19:27:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/18 19:27:35 INFO mlflow.store.db.utils: Updating database tables
2026/01/18 19:27:35 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/18 19:27:35 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/18 19:27:35 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/18 19:2

MLflow Run 'LASSO_Optimal' enregistré avec succès.


### 2.3 Conteneurisation (Docker)

Tâche 2.4 : Rôle des commandes Docker:

Le Dockerfile est un script contenant une série d'instructions permettant de construire automatiquement une image isolée contenant votre modèle et son environnement d'exécution.

FROM : Définit l'image de base (le système d'exploitation et l'environnement minimal, ex: Python).

COPY : Transfère les fichiers locaux (code, modèles .pkl) dans l'image Docker.

CMD : Définit la commande par défaut qui s'exécute au démarrage du conteneur (ex: lancer le serveur API).

In [14]:
%%writefile Dockerfile
# 1. Image de base légère avec Python
FROM python:3.9-slim

# 2. Définition du répertoire de travail dans le conteneur
WORKDIR /app

# 3. Copie du fichier des dépendances et installation
# requirements.txt doit contenir : scikit-learn, numpy, joblib, flask
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 4. Copie des fichiers locaux vers le conteneur
COPY lasso_model.pkl .
COPY scaler.pkl .
COPY app.py .

# 5. Commande de lancement de l'application
CMD ["python", "app.py"]

Writing Dockerfile


## Conclusion et Monitoring

Tâche 3.1 : Git, MLflow et Docker  Ce trio forme la base du CI/CD en ML :

L'intégration de ces trois outils est fondamentale pour créer des chaînes de CI/CD (Intégration et Déploiement Continus) robustes:

Git : Assure le versionnage du code source. Il permet la collaboration et garantit que chaque modification du pipeline peut être tracée et annulée si nécessaire.

MLflow : Gère le cycle de vie du modèle. Il permet de comparer les expériences (tracking) et de versionner les modèles eux-mêmes (Model Registry), assurant ainsi que l'on déploie toujours la version la plus performante.

Docker : Garantit la reproductibilité technique. Il élimine le problème du "ça marche sur ma machine" en emballant le modèle et ses dépendances dans un environnement immuable




Tâche 3.2 : Monitoring en production  Outre le RMSE, il faut surveiller :

Une fois le modèle en production, le RMSE ne suffit plus. Deux aspects cruciaux doivent être surveillés:


Data Drift (Dérive des données) : Il s'agit de surveiller si les propriétés statistiques des données d'entrée changent au fil du temps par rapport aux données d'entraînement (ex: changement de comportement des utilisateurs).

Concept Drift (Dérive du concept) : C'est le cas où la relation entre les variables d'entrée et la cible change (ex: une crise économique qui change brutalement la valeur des indicateurs de santé sans que les données des patients ne changent).